In [1]:
# import libraries
import requests 
import hashlib
from collections import defaultdict
import json
from groq import Groq
from dotenv import load_dotenv
import os
from tqdm.auto import tqdm
import pickle
import pandas as pd
import re

In [2]:
# get the data
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

# flatten the data
for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
# function that generates unique id for each document
def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [4]:
# create unique id for each document
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [5]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

In [6]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [7]:
len(hashes), len(documents)

(947, 948)

In [8]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [9]:
hashes['593f7569']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [10]:
# create data file with unique ids
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [11]:
# create a prompt template for LLM
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 
Dont say anything except what was tasked, avoid things like "Here are the 5 questions this student might ask based on the FAQ record"

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [12]:
# load environment variables
load_dotenv()

# create client calling Groq class
client = Groq(api_key=os.getenv('GROQ_API_KEY'))

In [13]:
# create a function that generates questions
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",
    )

    json_response = response.choices[0].message.content
    return json_response

In [14]:
# # generate questions for each document using LLM
# results = {}

# for doc in tqdm(documents): 
#     doc_id = doc['id']
#     if doc_id in results:
#         continue

#     questions = generate_questions(doc)
#     results[doc_id] = questions

In [15]:
# to save time we are reading in a preprocessed response
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [16]:
# clean up the results
def escape_json_strings(json_string):
    # Use regex to escape backslashes
    escaped_string = re.sub(r'\\', r'\\\\', json_string)
    return escaped_string

def update_results(results):
    for doc_id, json_string in results.items():
        try:
            escaped_string = escape_json_strings(json_string)
            results[doc_id] = json.dumps(json.loads(escaped_string))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for doc_id {doc_id}: {e}")
            results[doc_id] = None

# Problematic entries with manual fixes
fixed_entries = {
    'e3106e07': '[\n  "How can I access the course materials?",\n  "Is the course available for free?",\n  "Do I need to register to access the course?",\n  "Can I download the course videos?",\n  "What are the system requirements for the course?"\n]',
    '593a85ba': '[\n  "What is the course schedule?",\n  "When are the live sessions held?",\n  "Is there a calendar for the course?",\n  "How can I sync the course schedule with my calendar?",\n  "Are the live sessions recorded?"\n]',
    'b36ea564': '[\n  "What is the refund policy?",\n  "Can I get a refund if I am not satisfied?",\n  "How do I request a refund?",\n  "What is the deadline for requesting a refund?",\n  "Are there any fees for refund requests?"\n]',
    '11f7371c': '["What is the course duration?", "How long does the course last?", "Is there a time limit to complete the course?", "Can I extend the course duration?", "What happens if I need more time to finish the course?"]',
    '15f361b7': '[\n  "What is the cost of the course?",\n  "Are there any discounts available?",\n  "Can I get a scholarship for the course?",\n  "Is there a payment plan option?",\n  "What payment methods are accepted?"\n]'
}

# Update the full results dictionary with the fixed entries
results.update(fixed_entries)

# Run the update function on the full results dictionary
update_results(results)


In [17]:
# parse the results
parsed_results = {}

for doc_id, json_questions in results.items():
    parsed_results[doc_id] = json.loads(json_questions)


In [18]:
# index the documents by id
doc_index = {d['id']: d for d in documents}

In [19]:
# create a list of final results
final_results = []

for doc_id, questions in parsed_results.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [20]:
# create a dataframe
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

# save the dataframe
df.to_csv('ground-truth-data.csv', index=False)

In [23]:
df.head()

,question,course,document
0,When does the course begin?,data-engineering-zoomcamp,c02e79ef
1,How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
2,What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
3,How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
4,Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
